# IDENTIFYING CITIES TO INVEST IN RE

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import logging
import time

In [2]:
df = {}  
df_city_vals = pd.DataFrame()

In [3]:
def parse_city_data(city_state):
    city_values = pd.DataFrame()
    cols = ['city','population_2000', 'population_2017', 'pop_growth_%', 'median_income_2000', 'median_income_2017', 'income_growth_%',
            'median_house_2000', 'median_house_2017', 'median_house_growth_%', 
            'crime_index_2005', 'crime_index_2018', 'crime_increase_%', 'poverty_rate_%']

    city_link = get_city_address(city_state)    
    text = requests.get(city_link)
    soup = BeautifulSoup(text.content, 'html.parser')
    
    # get populataion section
    section_population = soup.find('section', {'id':'city-population'})
    # get median income and house text
    section_med_inc = soup.find('section', {'id':'median-income'})
    # get crime section
    section_crime_index = soup.find('section', {'id':'crime'})
    # get povery level
    section_poverty_level = soup.find('section', {'id':'poverty-level'})
    
    
    tag_txt_house = 'Estimated median house or condo value in 2017:'
    tag_txt_income = 'Estimated median household income in 2017:'
    tag_txt_population = 'Population in 2017:'
    tag_txt_poverty = 'Percentage of residents living in poverty in 2017:'
        
    tag = section_med_inc.find('b',text=tag_txt_house)
 
    city_values == {}
    all_fields = []
    # get population data 
    population_fields = get_population(tag_txt_population, section_population)
    # find and update the income values
    income_fields = get_city_values(tag_txt_income, section_med_inc)
    # find and update the median house values
    house_fields = get_city_values(tag_txt_house, section_med_inc)
    # find crimes index data
    crime_index_fields = get_crime_index(section_crime_index)
    # find crimes index data
    poverty_level_field = get_poverty_level(tag_txt_poverty,section_poverty_level)    

    all_fields.append(city_state)
    all_fields.extend(population_fields)
    all_fields.extend(income_fields)
    all_fields.extend(house_fields)
    all_fields.extend(crime_index_fields)
    all_fields.extend(poverty_level_field)
         
    global df_city_vals
    df_city_vals = df_city_vals.append(pd.DataFrame([all_fields], 
                                                    columns=cols),ignore_index = True)   
    
def get_city_values(srch_txt, section):
    tag = section.find('b',text=srch_txt)   
    latest_element = tag.next_sibling
    compare_element = tag.findNext('b').next_sibling
    compare_col = convert_to_num(compare_element)
    latest_col = convert_to_num(latest_element)    
    pct_diff =  get_pct(latest_col, compare_col)
    
    return [int(compare_col), int(latest_col), float(pct_diff)]

def get_population(srch_txt, section):
#     print(section)
    tag = section.find('b',text=srch_txt) 
    latest_col_txt = tag.next_sibling
    latest_col_txt = latest_col_txt.split('(')[0];
    latest_col = convert_to_num(latest_col_txt)    

    pct_diff = tag.findNext('b').next_sibling.split('%')[0]
    pct_diff = float(pct_diff)
    
    # poulation of latest year and % diff is provided, need to calculate what was the population
    compare_popln = latest_col/(1 + pct_diff/100);

#     compare_col = convert_to_num(compare_element)
#     latest_col = convert_to_num(latest_element)    
#     pct_diff =  get_pct(latest_col, compare_col)
    
    return [int(compare_popln), int(latest_col), float(pct_diff)]

def get_crime_index(section_crime):
    # if no crime data found for the city
    if section_crime == None:
        return [None, None, None]
    cr = section_crime.find_all('tr', {"class": "nosort"})[0]
    crimes = []
    for idx, val in enumerate(cr.find_all('td')):
        if idx != 0:
            crime = val.text.strip()
            crimes.append(float(crime))
    cr_pct_dff = get_pct(crimes[0], crimes[1])            
    return [crimes[0], crimes[-1], cr_pct_dff] 

def get_poverty_level(srch_txt, section_poverty):
    if section_poverty == None:
        return [None, None, None]
    tag = section_poverty.find('b',text=srch_txt)
    poverty_level_txt = tag.next_sibling
    poverty_level_txt = poverty_level_txt.split('%')[0];
    poverty_level = float(poverty_level_txt)     
    return [poverty_level] 

def convert_to_num(text):
    tex = text.replace(',', '')
    return int(re.search('[0-9]+',tex).group(0))

def get_pct(num1, num2):
    return round((num1/num2  - 1) * 100, 2)

def get_city_address(City_link):
    city_data_address = 'http://www.city-data.com'
    return city_data_address + '/city/' + City_link + '.html'

       

In [4]:
cities_data = 'data/large_market_database.csv'
df = pd.read_csv(cities_data)
df.head
cities = []
for city in df['Location']:
    #   concatenate city and state with '-', replace empty space with '-' and ' with -
    #   O'Fallon-Missouri to O-Fallon-Missouri
    cy = city.replace(' , ', '-').replace(' ','-').replace("'", '-').replace(',-','-').replace('--', '-')
    cities.append(cy)
df['Cities_ready'] = cities  
# drop cities containing "balance" 
# like"Augusta-Richmond-County-consolidated-government-(balance)-Georgia"
df_cy = df[~df.Cities_ready.str.contains("balance")]
citis_ser = df_cy.iloc[:]['Cities_ready']
citis_ser.head()


0          Flint-Michigan
1        Detroit-Michigan
2         Youngstown-Ohio
3            Gary-Indiana
4    Reading-Pennsylvania
Name: Cities_ready, dtype: object

In [ ]:

# #for testing
# data = np.array(['Round-Rock-Texas', "Flint-Michigan"])  
# citis_ser = pd.Series(data)

# parse_city_data('Miami-Gardens-Florida')        
for idx, city in enumerate(citis_ser):
    print(str(idx) + ' of ' + str(citis_ser.count()) + ': ' + city)
    try:
        parse_city_data(city)
        # protecting throtling from city-data
        time.sleep(2)
    except Exception as e:
        print('Data not found for: ' + city)
        print(str(e))
        pass
df_file = 'data/cities_market_data.csv'
df_city_vals.to_csv(df_file, index=False)
df_city_vals.head()  

0 of 547: Flint-Michigan


In [8]:
df_file = 'data/cities_market_data.csv'
df_city_vals.to_csv(df_file, index=False)